In [1]:
# Test out some data
import plotly
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

import plotly.express as px
from src.data.load import get_data


data_id = "solar"
df = get_data(data_id)
df.reset_index(inplace=True)

fig = px.scatter(data_frame=df, x="x", y="y")
plotly.offline.iplot(fig)

In [2]:
import tensorflow as tf
from src.models.gp import GP


tf.config.set_visible_devices([], "GPU")  # My M1 acting up
gp = GP()
x, y = df["x"].values.reshape(-1, 1), df["y"].values.reshape(-1, 1)
gp.fit(x, y, steps=0)
preds = gp.predict(x)

2022-11-03 10:27:21.698447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 10:27:24.299383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
import plotly.graph_objects as go
import numpy as np


def plot_mean_preds_with_ci(x_seen, y_seen, x_new, means, variances_, ci_in_sd: float = 2.) -> None:
    """Expects flat vectors."""
    sds = np.sqrt(variances_)
    ci_high = means + (ci_in_sd * sds)
    ci_low = means - (ci_in_sd * sds)
    fig = go.Figure([
        go.Scatter(
            x=x_seen,
            y=y_seen,
            name="Seen data",
            mode="markers",
        ),
        go.Scatter(
            x=x_new,
            y=means,
            name="Mean predictions",
            mode="lines",
        ),
        go.Scatter(
            x=np.concatenate([x_new, x_new[::-1]]), # x, then x reversed
            y=np.concatenate([ci_high, ci_low[::-1]]), # upper, then lower reversed
            fill="toself",
            name=f"{ci_in_sd}-SD confidence interval"
        )
    ])
    fig.show()

In [14]:
# Let's add confidence intervals

m, cov = gp.conditional_distribution_at(x)
clean_v = np.maximum(cov.diagonal(), 0)

In [15]:
plot_mean_preds_with_ci(x.flatten(), y.flatten(), x.flatten(), m.flatten(), clean_v.flatten())

In [16]:
# Let's drop some points

In [17]:
x_partial = x[::2]
y_partial = y[::2]

In [21]:
gp_partial = GP(lengthscale=3.)
gp_partial.fit(x_partial, y_partial, steps=0)
m, cov = gp_partial.conditional_distribution_at(x)
clean_v = np.maximum(cov.diagonal(), 0)
plot_mean_preds_with_ci(
    x_seen=x_partial.flatten(), 
    y_seen=y_partial.flatten(), 
    x_new=x.flatten(),
    means=m.flatten(), 
    variances_=clean_v.flatten(),
)

In [ ]:
# TODO: play around with optimisation, add noisy observations?